In [1]:
# Import Libraries
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.cluster import KMeans
from sklearn.metrics import mean_squared_error
import itertools
from sklearn.metrics import silhouette_samples, silhouette_score
%matplotlib inline


In [4]:
!pwd

/home/jovyan/work/ML_recipes.md/Projects


In [162]:
# Import the Movies dataset
#movies = pd.read_csv('ml-latest-small/movies.csv')
movies = pd.read_csv('./../Others/ml-1m/movies.dat'  , delimiter='::',header=None)# , index_col=0) 
movies.columns = ["movieId","Movie ", "genres"]

movies.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,movieId,Movie,genres
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [151]:
movieGenres =  movies['Genre'].str.split("|",expand=True).rename(columns = lambda x: "Genres"+str(x+1))
movieGenres.head()

,Genres1,Genres2,Genres3,Genres4,Genres5,Genres6
0,Animation,Children's,Comedy,None,None,None
1,Adventure,Children's,Fantasy,None,None,None
2,Comedy,Romance,None,None,None,None
3,Comedy,Drama,None,None,None,None
4,Comedy,None,None,None,None,None


In [152]:
movieGenres['Genres6'].value_counts()

War    1
Name: Genres6, dtype: int64

In [153]:
movieAll = movies.join(movieGenres)
movieAll.head()

,Movie ID,Movie,Genre,Genres1,Genres2,Genres3,Genres4,Genres5,Genres6
0,1,Toy Story (1995),Animation|Children's|Comedy,Animation,Children's,Comedy,None,None,None
1,2,Jumanji (1995),Adventure|Children's|Fantasy,Adventure,Children's,Fantasy,None,None,None
2,3,Grumpier Old Men (1995),Comedy|Romance,Comedy,Romance,None,None,None,None
3,4,Waiting to Exhale (1995),Comedy|Drama,Comedy,Drama,None,None,None,None
4,5,Father of the Bride Part II (1995),Comedy,Comedy,None,None,None,None,None


#### Rating data Set
UserID::MovieID::Rating::Timestamp

In [164]:
# UserID::MovieID::Rating::Timestamp
# Import the Ratings dataset
ratings = pd.read_csv('./../Others/ml-1m/ratings.dat'  , delimiter='::',header=None) 
ratings.columns = ['userId','movieId','rating','timestamp']
#ratings.index.name='UserID"
ratings.head()

/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [31]:
ratings.shape

(1000209, 3)

In [140]:
# Import the users dataset
# UserID::Gender::Age::Occupation::Zip-code
users = pd.read_csv('./../Others/ml-1m/users.dat'  , delimiter='::',header=None, index_col=0) 
users.columns = ['Gender','Age','Occupation','Zip-code']
users.index.name="UserId"
users.head()


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  This is separate from the ipykernel package so we can avoid doing imports until


,Gender,Age,Occupation,Zip-code
UserId,,,,
1,F,1,10,48067
2,M,56,16,70072
3,M,25,15,55117
4,M,45,7,02460
5,M,25,20,55455


In [141]:
# Print the number of records and the total number of movies
print('The dataset contains: ', len(ratings), ' ratings of ', len(movies), ' movies.', ' by ',len(users),'users')

The dataset contains:  1000209  ratings of  3883  movies.  by  6040 users


In [172]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [182]:
sciFi_genre="Sci-Fi"
movies[movies['genres'].str.contains(sciFi_genre)].head()

,movieId,Movie,genres
23,24,Powder (1995),Drama|Sci-Fi
28,29,"City of Lost Children, The (1995)",Adventure|Sci-Fi
31,32,Twelve Monkeys (1995),Drama|Sci-Fi
65,66,Lawnmower Man 2: Beyond Cyberspace (1996),Sci-Fi|Thriller
75,76,Screamers (1995),Sci-Fi|Thriller


In [170]:
ratings[ratings['movieId'].isin(genre_movies['movieId'])]

,userId,movieId,rating,timestamp
22,1,1270,5,978300055
26,1,1097,4,978301953
44,1,260,4,978300760
60,2,2628,3,978300051
62,2,2916,3,978299809
64,2,1210,4,978298151
78,2,1253,3,978299120
87,2,1544,4,978300174
97,2,3471,5,978298814
98,2,589,4,978299773


In [169]:

#print(ratings.shape)
#print(movies.shape)
genres =['Romance', 'Sci-Fi']
column_names=['avg_romance_rating', 'avg_scifi_rating']
genre_ratings = pd.DataFrame()

for genre in genres:   
    print(genre)
    genre_movies = movies[movies['genres'].str.contains(genre) ]
#    print(genre_movies)
    avg_genre_votes_per_user = ratings[ratings['movieId'].isin(genre_movies['movieId'])].loc[:, ['userId', 'rating']].groupby(['userId'])['rating'].mean().round(2)        
    genre_ratings = pd.concat([genre_ratings, avg_genre_votes_per_user], axis=1)
        
genre_ratings.columns = column_names

genre_ratings.head()



Romance
Sci-Fi


,avg_romance_rating,avg_scifi_rating
userId,,
1,3.67,4.33
2,3.71,3.59
3,3.80,3.83
4,4.00,3.56
5,3.10,3.07
